# Tarea 3
En esta tarea usted deberá completar el siguiente notebook, en el cual va a implementar el protocolo ElGamal sobre curvas elípticas.

## Funciones auxiliares
Primero necesitamos un test de primalidad, para lo cual usamos lo mismo que para la pregunta 2 de la Tarea 2.

In [ ]:
import random

def _is_natural_power(n):
    # Para cada posible exponente, hacemos búsqueda binaria de la base
    search_exponent = 2
    
    # Optimiazación: si n no es a ^ k no puede ser a ^ (kr) para ningún
    # r, por lo que sólo probamos con exponentes primos
    avoid_exponents = set()
    
    while pow(2, search_exponent) <= n:
        
        if search_exponent not in avoid_exponents:
            # Usamos búsqueda binaria "creciente" para definir el intervalo
            # inicial
            search_start = 2
            i = 2
            while search_start ** search_exponent < n:
                search_start *= 2
                avoid_exponents.add(search_exponent * i)
                i += 1
                
            upper = search_start
            lower = search_start // 2

            # Búsqueda binaria
            while lower != upper:
                mid = (upper + lower) // 2
                result = pow(mid, search_exponent)
                if result < n:
                    lower = mid + 1
                elif result > n:
                    upper = mid
                else:
                    return True

            # Caso borde en que upper ^ search_exponent era justo n
            if pow(upper, search_exponent) == n:
                return True
            
        search_exponent += 1
    
    return False

In [ ]:
def _extended_euclid(a, b):
    if a > b:
        return _extended_euclid_base(a, b)
    r, s, t = _extended_euclid_base(b, a)
    return r, t, s

def _extended_euclid_base(a, b):
    prev_r, r = a, b
    prev_s, s = 1, 0
    prev_t, t = 0, 1

    while r != 0:
        q = prev_r // r
        prev_r, r = r, prev_r % r
        prev_s, s = s, prev_s - q * s
        prev_t, t = t, prev_t - q * t

    return prev_r, prev_s, prev_t

In [ ]:
def _is_probably_prime(n, iterations=100):
    if n == 2:
        return True
    if n % 2 == 0 or n == 1:
        return False
    if _is_natural_power(n):
        return False
    
    for i in range(iterations):
        a = random.randint(1, n - 1)
        if _extended_euclid(a, n)[0] > 1:
            return False
        b = pow(a, (n - 1) // 2, n)
        if b == n - 1:
            found_negative = True
        elif b != 1:
            return False
    
    return found_negative

## Una primera clase y sus elementos
Como un ejemplo de la forma en la cual debe ser implementado el protocolo ElGamal en esta tarea, consideramos a los grupos Z<sub>p</sub><sup>\*</sup> vistos en clases. En particular, definimos la clase `ZpStar` cuyas instancias son estos grupos. Para representar a los elementos dentro de Z<sub>p</sub><sup>\*</sup>, en su constructor se crea dinámicamente otra clase.

In [ ]:
class ZpStar:
    
    def __init__(self, p):
        if not _is_probably_prime(p):
            raise Exception(f"p={p} is not a prime number")
        class Element:
            def __init__(self, value):
                if value < 1 or value > p-1:
                    raise Exception(f"value={value} is not in the range 1,...,{p-1}")
                self.value = value

            # Allows to compare elements with ==
            def __eq__(self, other_element):
                return self.value == other_element.value

            # Allows to operate elements with *
            def __mul__(self, other_element):
                return Element((self.value * other_element.value) % p)

            # Allows to use ** as exponentiation
            def __pow__(self, exponent):
                return Element(pow(self.value, exponent, p))

            # Allows to use str(e) to transform element e into a string
            def __str__(self):
                return str(self.value)
                
        self.element_class = Element
                
    def get_identity(self):
        return self.get_element(1)
    
    def get_element(self, n):
        return self.element_class(n)

Ahora usted debe completar las definiciones de las clases para enviar y recibir mensajes con ElGamal.

In [ ]:
class ElGamalReceiver:
    def __init__(self, group, generator, group_order):
        # Is the order of the generator correct? For this we check that
        # 1. The group order is prime; and
        # 2. The generator to the power of the order is 1.
        ##### POR COMPLETAR 
            
        # The secret key is simply a random scalar in the correct range
        ##### POR COMPLETAR

    def get_public_key(self):
        # The public key must be a tuple containing the group,
        # the generator, the order of the generator, and the
        # generator to the power secret_key
        ##### POR COMPLETAR 
        
    def decrypt(self, ciphertext):
        # Receives a pair representing a ciphertext.
        ##### POR COMPLETAR 

De la misma forma que para la clase `ZpStar`, su implementación del constructor de la clase `ElGamalReceiver` debe generar excepciones si los parámetros entregados no son correctos (puede suponer que los tipos de estos parámetros siempre van a ser los correctos). Por ejemplo, si `group_order` no es un número primo, entonces se debe generar una excepción (puede suponer que el valor entregado en `group_order` va a ser un número natural mayor o igual a 1).

In [ ]:
class ElGamalSender:
    def __init__(self, pubkey):
        ##### POR COMPLETAR 

    def encrypt(self, group_element):    
        # We are only encrypting single group elements.
        # According to ElGamal, the ciphertext is a pair.
        ##### POR COMPLETAR    

Recuerde que el protocolo ElGamal está definido para cualquier grupo, y por lo tanto se espera que su implementación funcione con una interfaz genérica para interactuar con estos objetos. Por ejemplo, en el siguiente código se utiliza el protocolo ElGamal para el grupo Z<sub>149</sub><sup>\*</sup>.

In [ ]:
p, generator, order = 149, 25, 37
group = ZpStar(p)
generator = group.get_element(generator)

receiver = ElGamalReceiver(group, generator, order)
sender = ElGamalSender(receiver.get_public_key())

plaintext = group.get_element(121)
ciphertext = sender.encrypt(plaintext)
dec = receiver.decrypt(ciphertext)

print("\nPlaintext:      ", str(plaintext))
print("Decrypted text: ", str(dec))

Una vez que haya completado las definiciones de las clases para enviar y recibir mensajes con ElGamal, el código anterior debe mostrar lo siguiente:

Plaintext:       121  
Decrypted text:  121

Nótese que en este caso 121 es el mensaje a enviar, el cual es definido como un elemento del grupo Z<sub>149</sub><sup>\*</sup> a través de la línea `plaintext = group.get_element(121)`.

Como un segundo ejemplo considere un grupo Z<sub>p</sub><sup>\*</sup> que es usado en la práctica.

In [ ]:
p = 17125458317614137930196041979257577826408832324037508573393292981642667139747621778802438775238728592968344613589379932348475613503476932163166973813218698343816463289144185362912602522540494983090531497232965829536524507269848825658311420299335922295709743267508322525966773950394919257576842038771632742044142471053509850123605883815857162666917775193496157372656195558305727009891276006514000409365877218171388319923896309377791762590614311849642961380224851940460421710449368927252974870395873936387909672274883295377481008150475878590270591798350563488168080923804611822387520198054002990623911454389104774092183
generator_value = 8041367327046189302693984665026706374844608289874374425728797669509435881459140662650215832833471328470334064628508692231999401840332046192569287351991689963279656892562484773278584208040987631569628520464069532361274047374444344996651832979378318849943741662110395995778429270819222431610927356005913836932462099770076239554042855287138026806960470277326229482818003962004453764400995790974042663675692120758726145869061236443893509136147942414445551848162391468541444355707785697825741856849161233887307017428371823608125699892904960841221593344499088996021883972185241854777608212592397013510086894908468466292313
order = 63762351364972653564641699529205510489263266834182771617563631363277932854227

group = ZpStar(p)
generator = group.get_element(generator_value)

receiver = ElGamalReceiver(group, generator, order)

sender = ElGamalSender(receiver.get_public_key())

message = 98983374938374643576429837455646547364648570928735482734692838743

plaintext = group.get_element(message)
ciphertext = sender.encrypt(plaintext)
dec = receiver.decrypt(ciphertext)

print("\nPlaintext:      ", str(plaintext))
print("Decrypted text: ", str(dec),"\n")

Una vez que haya completado las definiciones de las clases para enviar y recibir mensajes con ElGamal, el código anterior debe mostrar lo siguiente:

Plaintext:       98983374938374643576429837455646547364648570928735482734692838743  
Decrypted text:  98983374938374643576429837455646547364648570928735482734692838743

## Utilizando curvas elípticas
En esta segunda parte de la tarea, usted debe utilizar el protocolo ElGamal sobre grupos definidos por curvas elípticas. En particular, debe completar la siguiente definición de la clase `EllipticCurve` considerando la definición de curvas elípticas dada en la ecuación (9.2) del la sección 9.3.4 del texto guía del curso:

Jonathan Katz y Yehuda Lindell. Introduction to Modern Cryptography. Chapman and Hall/CRC,
tercera edición, 2020.

In [ ]:
class EllipticCurve:
    def __init__(self, A, B, p):
        # Raise exceptions
        ##### POR COMPLETAR         
        
        class Element:
            def __init__(self, x, y = None):
                ##### POR COMPLETAR  

            def __eq__(self, other_element):
                ##### POR COMPLETAR  

            def __mul__(self, other_element):
                ##### POR COMPLETAR  
                    
            def __pow__(self, exponent):
                ##### POR COMPLETAR  

            def __str__(self):
                ##### POR COMPLETAR  
                    
        self.element_class = Element
                
    def get_identity(self):
        ##### POR COMPLETAR 
                    
    def get_element(self, x, y):
        ##### POR COMPLETAR 

En esta definición de `EllipticCurve`, dado un número primo `p`, cada punto sobre la curva es un par ordenado `(x,y)` con `x` e `y` en el conjunto `{0, ..., p-1}`, excepto por el neutro del grupo que un elemento especial que no necesita notación de par ordenado (ver el libro de Jonathan & Lindell para una explicación de esto). Por esto el constructor de la clase `EllipticCurve` recibe dos argumentos para representar un par ordenado, y también considera el caso en que `y` no esté definido porque se está utilizando el elemento neutro.

De la misma forma que para la clase `ZpStar`, su implementación del constructor de la clase `EllipticCurve` debe generar excepciones si los parámetros entregados no son correctos (puede suponer que los tipos de estos parámetros siempre van a ser los correctos). Por ejemplo, si `p` no es un número primo, entonces se debe generar una excepción.

Su definición de la clase `EllipticCurve` va a ser utilizada por la implementación del protocolo ElGamal de la misma forma que para la clase `ZpStar`. Por ejemplo, en el siguiente código se utiliza el protocolo ElGamal para la curva elíptica [P-256](https://neuromancer.sk/std/nist/P-256).

In [ ]:
    p = 115792089210356248762697446949407573530086143415290314195533631308867097853951
    A = 115792089210356248762697446949407573530086143415290314195533631308867097853948
    B = 41058363725152142129326129780047268409114441015993725554835256314039467401291
    g_x = 48439561293906451759052585252797914202762949526041747995844080717082404635286
    g_y = 36134250956749795798585127919587881956611106672985015071877198253568414405109
    q = 115792089210356248762697446949407573529996955224135760342422259061068512044369

    group = EllipticCurve(A, B, p)
    g = group.get_element(g_x, g_y)

    receiver = ElGamalReceiver(group, g, q)

    sender = ElGamalSender(receiver.get_public_key())

    message_x = 3649244856384847635638847363849074342342433643773
    message_y = 36810392828448194526040058211987909976903679270241111391326603075746535787758
    plaintext = group.get_element(message_x, message_y)

    ciphertext = sender.encrypt(plaintext)

    dec = receiver.decrypt(ciphertext)

    print("\nPlaintext:      ", str(plaintext))
    print("Decrypted text: ", str(dec))

Dadas las clases para enviar y recibir mensajes con ElGamal implementadas en la primer parte de la tarea, el código anterior debe mostrar lo siguiente:

Plaintext:       (3649244856384847635638847363849074342342433643773, 368103928284481945260400582119879099769036792702
41111391326603075746535787758)  
Decrypted text:  (3649244856384847635638847363849074342342433643773, 368103928284481945260400582119879099769036792702
41111391326603075746535787758)

Nótese que en este caso

(3649244856384847635638847363849074342342433643773, 36810392828448194526040058211987909976903679270241111391326603075746535787758)

es el mensaje a enviar, el cual es definido como un elemento del grupo a través de la línea `plaintext = group.get_element(message_x, message_y)`

¡Buena suerte con la tarea!